In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/aaaaa/

/content/drive/MyDrive/aaaaa


In [3]:
!ls

kaggle.json  sampleSubmission.gz  test.gz     train.csv  train_v2.csv
rtrain.csv   test.csv		  train2.csv  train.gz	 tree.dot


In [4]:
features = ['id', 'state', 'C22', 'C1', 'banner_pos', 'device_id', 'device_ip',
      'site_category', 'app_id', 'app_domain', 'app_category', 'C23',
       'C24', 'device_model', 'device_type', 'device_conn_type', 'C14',
      'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

In [ ]:
df.columns

In [6]:
## in import les les Biblios
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd


n_rows = 300000
df = pd.read_csv("rtrain.csv", nrows=n_rows)


X = df.drop(['state', 'id', 'C22', 'C23', 'C24'], axis=1).values
Y = df['state'].values

## pour commencer on va 

n_train = 100000
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X_train_enc = enc.fit_transform(X_train)

X_test_enc = enc.transform(X_test)

# # avec la scikit-learn 
from sklearn.linear_model import SGDClassifier
sgd_lr = SGDClassifier(loss='log', penalty=None, fit_intercept=True, max_iter=100, learning_rate='constant', eta0=0.01)
sgd_lr.fit(X_train_enc.toarray(), Y_train)

pred = sgd_lr.predict_proba(X_test_enc.toarray())[:, 1]
print(f'Training samples: {n_train}, AUC on testing set: {roc_auc_score(Y_test, pred):.3f}')


# ---------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------------------

## la selection de features en utilisant la L1 regularization
sgd_lr_l1 = SGDClassifier(loss='log', penalty='l1', alpha=0.0001, fit_intercept=True, max_iter=100, learning_rate='constant', eta0=0.01)
sgd_lr_l1.fit(X_train_enc.toarray(), Y_train)

coef_abs = np.abs(sgd_lr_l1.coef_)
print(coef_abs)


## les 10 features les moins utilisables 
print(np.sort(coef_abs)[0][:10])

feature_names = enc.get_feature_names()
bottom_10 = np.argsort(coef_abs)[0][:10]
print('les10 features les moins importantes sont :\n', feature_names[bottom_10])


## les 10 meilleurs features 
print(np.sort(coef_abs)[0][-10:])
top_10 = np.argsort(coef_abs)[0][-10:]
print('les10 features les plus importantes sont ::\n', feature_names[top_10])



Training samples: 100000, AUC on testing set: 0.732
[[0.         0.13803421 0.         ... 0.         0.         0.13455693]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
les10 features les moins importantes sont :
 ['x0_1001' 'x8_84dda655' 'x8_84c2f017' 'x8_84ace234' 'x8_84a9d4ba'
 'x8_84915a27' 'x8_8441e1f3' 'x8_840161a0' 'x8_83fbdb80' 'x8_83fb63cd']
[0.6546022  0.74700015 0.76492194 0.81732684 0.92023462 0.99551065
 0.99600743 1.06115632 1.09704979 1.33308764]
les10 features les plus importantes sont ::
 ['x7_cef3e649' 'x3_7687a86e' 'x18_61' 'x18_15' 'x5_5e3f096f' 'x5_9c13b419'
 'x2_d9750ee7' 'x2_763a42b5' 'x3_27e3c518' 'x5_1779deee']


In [7]:

# dans cette partie on va aborder le Online learning


##      ----------          ----------            ----------
##     |  Model 1 |        |  model 2 |   --->   |  Model X |
##      ----------          ----------            ----------
##          ^                    ^                    ^ 
##          |                    |                    |
##
##      une dataset 1       une dataset 2       une dataset X
## 
## c'est pas vraiment la bonne demonstration ça mais ça fonctionne Qmm 


## dans ce cas là on augmente le nombre de samples à 1 million pour voir si ça fonctione avec un dataset si grand

n_rows = 100000 * 11
df = pd.read_csv("train.csv", nrows=n_rows)



df.columns = features

X = df.drop(['state', 'id', 'C22', 'C23', 'C24'], axis=1).values
Y = df['state'].values



n_train = 100000 * 10
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train)


# le nombre d'iterations est mis à 1 si on utilise  partial_fit.
sgd_lr_online = SGDClassifier(loss='log', penalty=None, fit_intercept=True, max_iter=1, learning_rate='constant',eta0=0.01)

import timeit
start_time = timeit.default_timer()


# On utilise les premiers 1,000,000 samples pour le training, et le 100,000 qui rest pour le testing
for i in range(10):
    x_train = X_train[i*100000:(i+1)*100000]
    y_train = Y_train[i*100000:(i+1)*100000]
    x_train_enc = enc.transform(x_train)
    sgd_lr_online.partial_fit(x_train_enc.toarray(), y_train, classes=[0, 1])

print(f"--- {(timeit.default_timer() - start_time)}.3fs secondes ---")

x_test_enc = enc.transform(X_test)

pred = sgd_lr_online.predict_proba(x_test_enc.toarray())[:, 1]
print(f'samples d entrainement: {n_train * 10}, AUC sur le Dataset TEST: {roc_auc_score(Y_test, pred):.3f}')

## ============================= ====================== ================= ============== ##
# la classification en utilisant la regression logistique 

from sklearn import datasets
digits = datasets.load_digits()
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
Y = digits.target

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV
parameters = {'penalty': ['l2', None],
              'alpha': [1e-07, 1e-06, 1e-05, 1e-04],
              'eta0': [0.01, 0.1, 1, 10]}

sgd_lr = SGDClassifier(loss='log', learning_rate='constant', eta0=0.01, fit_intercept=True, max_iter=100)

grid_search = GridSearchCV(sgd_lr, parameters, n_jobs=-1, cv=5)

grid_search.fit(X_train, Y_train)
print(grid_search.best_params_)

sgd_lr_best = grid_search.best_estimator_
accuracy = sgd_lr_best.score(X_test, Y_test)
print(f'precision sur le dataset TEST: {accuracy*100:.1f}%')


--- 91.41519984199999.3fs secondes ---
samples d entrainement: 10000000, AUC sur le Dataset TEST: 0.762
{'alpha': 1e-05, 'eta0': 10, 'penalty': None}
precision sur le dataset TEST: 96.1%
